In [71]:
from qutip import *
from scipy.optimize import minimize
import numpy as np
from supports import *
from plotsettings import *

In [82]:
# Device 
Nq = 10         # number of levels in qubit
Nc = 10         # number of levels in resonator 1
wc = 4.0928  # bare resonator frequency
Ec = 0.15  # anharmonicity (charging energy)
# wq = 6.811  # bare resonator frequency

wq_dressed_meas = 6.811
wc_dressed_meas = 4.0753

wq_list  = np.linspace(6.7, 6.9, 21)
g_list = np.linspace(0.16, 0.19, 31)
Ec_list = np.linspace(0.13, 0.16, 31)

b = tensor(destroy(Nq), qeye(Nc))
a = tensor(qeye(Nc), destroy(Nq))
nq = b.dag()*b
nc = a.dag()*a

In [83]:
wq_dressed = np.zeros((len(wq_list), len(g_list), len(Ec_list)))
wc_dressed = np.zeros((len(wq_list), len(g_list), len(Ec_list)))
anharmonicity = np.zeros((len(wq_list), len(g_list), len(Ec_list)))

for i, wq in enumerate(wq_list):
    for j, g in enumerate(g_list):
        for k, Ec in enumerate(Ec_list):
            Hi = (wq+Ec)*nq + wc*nc - Ec/12*(b + b.dag())**4  # intrinsic
            Hc = g*(a + a.dag())*(b + b.dag())  # coupling
            EE = (Hi+Hc).eigenenergies()
            wc_dressed[i][j][k] = EE[1]-EE[0]
            wq_dressed[i][j][k] = EE[2]-EE[0]
            anharmonicity[i][j][k] = 2*(EE[2]-EE[0])-(EE[6]-EE[0])

# wq_dressed = np.zeros((len(g_list), len(Ec_list)))
# wc_dressed = np.zeros((len(g_list), len(Ec_list)))
# anharmonicity = np.zeros((len(g_list), len(Ec_list)))

# for j, g in enumerate(g_list):
#     for k, Ec in enumerate(Ec_list):
#         Hi = (wq+Ec)*nq + wc*nc - Ec/12*(b + b.dag())**4  # intrinsic
#         Hc = g*(a + a.dag())*(b + b.dag())  # coupling
#         EE = (Hi+Hc).eigenenergies()
#         wc_dressed[j][k] = EE[1]-EE[0]
#         wq_dressed[j][k] = EE[2]-EE[0]
#         anharmonicity[j][k] = 2*EE[2]-EE[6]

In [84]:
wq_dressed_squared = np.multiply((wq_dressed-wq_dressed_meas), (wq_dressed-wq_dressed_meas))
wc_dressed_squared = np.multiply((wc_dressed-wc_dressed_meas), (wc_dressed-wc_dressed_meas))
anharmonicity_squared = np.multiply((anharmonicity-0.15), (anharmonicity-0.15))
diffs =  wq_dressed_squared + wc_dressed_squared + anharmonicity_squared

In [85]:
I, J, K = np.unravel_index(np.argmin(diffs, axis=None), diffs.shape)
wq = wq_list[I]
# J, K = np.unravel_index(np.argmin(diffs, axis=None), diffs.shape)
g = g_list[J]
Ec = Ec_list[K]
print(wq)
print(g)
print(Ec)

6.8100000000000005
0.185
0.138


In [86]:
Hi = (wq+Ec)*nq + wc*nc - Ec/12*(b + b.dag())**4  # intrinsic
Hc = g*(a + a.dag())*(b + b.dag())  # coupling
EE = (Hi+Hc).eigenenergies()
print(EE - EE[0])

[ 0.          4.07674087  6.81340298  8.15347332 10.88779762 12.23019731
 13.47707187 14.96217637 16.30691281 17.548377   19.03653916 19.98088506
 20.38361981 21.61965228 23.1108859  24.04805636 24.46031826 25.69089762
 26.31470518 27.18521651 28.11518887 28.53700889 29.76211297 30.37713322
 31.25953092 32.18228499 32.48255962 32.61388729 33.83329828 34.43970955
 35.33383234 36.24934726 36.54608771 36.72215306 37.90445353 38.50245164
 38.56329881 39.40881633 40.31637842 40.61073403 41.97559065 42.5653762
 42.65301622 43.42286476 44.3833815  44.6764637  44.87133491 46.04437971
 46.62849903 46.74385957 48.45030016 48.74323911 49.0067192  49.95437569
 50.69183383 50.83566038 52.21438543 52.50626722 52.81102097 53.14015681
 54.75493281 54.9282793  56.30185761 56.34050353 56.87975552 57.27183457
 58.79010928 59.02160001 60.46710435 60.94763424 61.19764583 61.40191712
 62.46057908 63.11546703 64.5941419  64.96017707 65.35447597 65.53055083
 67.20564615 68.49304162 68.72157718 69.51063463 69.

In [105]:
def model(params):
    wq, g, Ec = params
    Hi = (wq+Ec)*nq + wc*nc - Ec/12*(b + b.dag())**4  # intrinsic
    Hc = g*(a + a.dag())*(b + b.dag())  # coupling
    EE = (Hi+Hc).eigenenergies()
    wq_dressed = EE[2]-EE[0]
    wc_dressed = EE[1]-EE[0]
    anharmonicity = 2*(EE[2]-EE[0])-(EE[6]-EE[0])
    return (wq_dressed-wq_dressed_meas)**2 + (wc_dressed-wc_dressed_meas)**2 + (anharmonicity-0.15)**2

result = minimize(model, [wq, g, Ec])

In [106]:
print(result)
print("")
wq = result.x[0]
g  = result.x[1]
Ec = result.x[2]
Hi = (wq+Ec)*nq + wc*nc - Ec/12*(b + b.dag())**4  # intrinsic
Hc = g*(a + a.dag())*(b + b.dag())  # coupling
EE = (Hi+Hc).eigenenergies()
print(EE - EE[0])

      fun: 1.874839273540713e-14
 hess_inv: array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])
      jac: array([-5.32680356e-09, -4.76938935e-08, -6.51280830e-08])
  message: 'Optimization terminated successfully.'
     nfev: 5
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([6.80678739, 0.19303658, 0.13835391])

[ 0.          4.07530013  6.81099999  8.15059022 10.88373594 12.22587022
 13.47200001 14.95645293 16.3011401  17.54135768 19.02915085 19.97286889
 20.37639982 21.61067968 23.1018296  24.03770483 24.45164936 25.67996592
 26.30349403 27.17448907 28.10249453 28.52688961 29.7492163  30.36314827
 31.24712918 32.16724114 32.46808653 32.6023522  33.81843076 34.42298051
 35.31975405 36.23194804 36.52896871 36.71193307 37.88760933 38.48301343
 38.54580706 39.39318853 40.29661882 40.59118104 41.95676739 42.54326856
 42.63525224 43.40027259 44.36125751 44.65467777 44.85072406 46.02314565
 46.60376617 46.72602584 48.42579138 48.71940945 48.98981954 49.91777507

In [103]:
6.81099999*2- 13.47200001

0.14999996999999965

In [104]:
0.19/0.179

1.0614525139664805